In [1]:
import sys
import os
from  pathlib import  Path
sys.path.append(str(Path.cwd().parent))
from  src.clinfoai.pubmed_engine import PubMedNeuralRetriever
from dotenv import load_dotenv
load_dotenv()

Using reranking model: ncbi/MedCPT-Cross-Encoder
Loaded model: ncbi/MedCPT-Cross-Encoder
Loaded tokenizer: ncbi/MedCPT-Cross-Encoder
Threshold: 0.5


True

# Using Clinfo.AI 

In this tutorial, we will go through each step of the Clinfo.AI workflow. Before we start, we need to set up a few things. 


### 1.- Setting up enviorment:
1.a.- Install the conda enviroment using the yml file provided.

``` conda env create -f environment.yaml ```

1.b.- Select your environment to run notebook. I recommend using VScode: 



### 2.- Creating Accounts

You will need at least one account and at most two (depending on how many calls/hour you plan to do):
* OPENAI account: If you start a free account for the first time, you will get $5 in API credits.
* NCBI_API_KEY: This is only necessary if you plan to make more than 10 calls per hour.


Once you have created both accounts  go to **src\config.py** file and: 

* Set OPENAI_API_KEY to your openAI API key

If you created an NCBI API account add your key and email in the following values: 
* NCBI_API_KEY 
* EMAIL
* 
Otherwise, leave them as None

### 3.- Defining your own promts:
We have designed prompts for each step of Clinfo.ai Workflow, leaveriging the power of in-contex-learning. If you want to us your own promps you can edit them **src\prompts** otherwise we will use the default prompts:

In [2]:
PROMPS_PATH = os.path.join("src","clinfoai","prompts","PubMed","Architecture_1","master.json")

### 4.- Define Clinfo.AI LLM Backbone
Clinfo uses a chain of LLMs to summarize information, thus we need to define an LLM backbone. 

We will start with OpenAI models, however, if you have access to GPUs it is possible to use Clinfo.AI with vLLM to use OpenSource LLMs as backbones (check tutorial 3).

In [3]:
MODEL:str  = "gpt-3.5-turbo"
#MODEL:str = "Qwen/Qwen2-beta-7B-Chat"

### 5.- Init Clinfo+Pubmed Engnie
We have all the necessary data to start our clinfo+pubmed instance:

In [4]:
## 5.- Init Neural Retriever from path. 
# Do not change the path if you want to use base  prompts, otherwise specify your own prompt architecture
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")
EMAIL         = os.getenv("EMAIL")
nrpm = PubMedNeuralRetriever(
    architecture_path = PROMPS_PATH,
    model             = MODEL,
    verbose           = False,
    debug             = False,
    open_ai_key       = OPENAI_API_KEY,
    email             = EMAIL)


Task Name: pubmed_query_prompt
------------------------------------------------------------------------

Task Name: relevance_prompt
------------------------------------------------------------------------

Task Name: summarization_prompt
------------------------------------------------------------------------

Task Name: synthesize_prompt
------------------------------------------------------------------------


# Let's start!

In [5]:
### Step 0 : Ask a question ###
QUESTION    = "What is the prevalence of COVID-19 in the United States?"
QUESTION    = "What tests are needed to diagnose Chronic Neutropenia?"


## STEP 1 (Search PubMed): Convert the question into a query using an LLM
# This returns a list of queries (containing MESH terms)
# These queries are used to retrieve articles from NCBI
# Once retrieved we collect a list article ids.
pubmed_queries, article_ids = nrpm.search_pubmed(
    question=QUESTION,
    num_results=10,
    num_query_attempts=1)

print(f"Articles retrived: {len(article_ids)}")
print(pubmed_queries)
print(article_ids)

/Users/aaron.dinh.int/Library/Caches/pypoetry/virtualenvs/clinfo-ai-vatf6MVj-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/aaron.dinh.int/Library/Caches/pypoetry/virtualenvs/clinfo-ai-vatf6MVj-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Articles retrived: 10
['("Chronic Neutropenia" AND "diagnosis" AND "tests")']
['10388004', '27841775', '6602565', '19305028', '3534197', '20301576', '24827415', '30870474', '34303547', '11964321']


In [6]:
## Step 2: Fetch article data
# Preiously, we only extracted he PMIDs. No we will use those  PMIDs to retrive the metadata:
articles = nrpm.fetch_article_data(article_ids)

# Print example for first article: 
article_num = 1
print(f"Article {article_num}:\n")

#print(articles[article_num].keys())
#print(articles[article_num]['PubmedData'])
print(articles[article_num]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"])
#print(articles[article_num]["MedlineCitation"]["Article"])


Article 1:

[StringElement('Neutropenia lasting for at least for 3 months and not attributable to drugs or a specific genetic, infectious, inflammatory, autoimmune or malignant cause is called chronic idiopathic neutropenia (CIN). CIN and autoimmune neutropenia (AIN) are very similar and overlapping conditions. The clinical consequences depend upon the severity of neutropenia, but it is not considered a premalignant condition.', attributes={'Label': 'PURPOSE OF REVIEW'}), StringElement('Long-term observational studies in children indicate that the disease often lasts for 3-5 years in children, then spontaneously remits, but it rarely remits in adult cases. The value of antineutrophil antibody testing in both children and adults is uncertain. Most recent data suggest that CIN and AIN are immune-mediated diseases, but there are no new clinical or genetic tests to aid in diagnosis. Treatment with granulocyte colony stimulating factor (G-CSF) is effective to increase blood neutrophils in a

In [7]:
# STEP 3 Summarize each article
# This step is parallelized, though it might look like one single call, it performs one call per article to summarize.
# Then the relevancy of the article (based on the original question) is provided by another LLM call.

article_summaries,irrelevant_articles =  nrpm.summarize_each_article(articles, QUESTION)

In [8]:
# Summaries for relevant articles
article_summaries

[{'title': '[Neutropenia in infancy - sometimes chronic and benign - own experiences].',
  'url': 'https://pubmed.ncbi.nlm.nih.gov/19305028/',
  'abstract': 'Chronic neutropenia is a decrease in circulating neutrophils in the peripheral blood lasting over 6 months. Values need to be refered with the age and race. In children aged 2 weeks to 12 months reffered values are above 1000/03BCL. There are congenital and aquired reasons of neutropenia in infancy. The most common type of chronic neutropenia in infants is chronic, benign neutropenia (AIN). Authors present ten infants between three and six months with chronic, benign neutropenia. The reason of ordering laboratory tests at outpatient clinic were benign upper respiratory tract infections (four cases), pallor (four cases) and on parental demand (one case). In one infant neutropenia was observed during treatment of pneumonia at a district hospital.',
  'citation': 'Kaczorowska-Hać B, Wierzba J, Stefanowicz J, Sielachowicz K, Wlazłowsk

In [9]:
# Articles deemed irelevant
irrelevant_articles 

[{'title': 'Physician Education: Myelodysplastic Syndrome.',
  'url': 'https://pubmed.ncbi.nlm.nih.gov/10388004/',
  'abstract': 'CHARACTERISTICS AND PATHOLOGY OF MYELODYSPLASTIC SYNROME: Myelodysplastic syndrome (MDS) is a disease of the blood whose etiology is unclear. There is little that can be done therapeutically, and the prognosis for patients with this disease is poor. The main hematologic finding is anemia, but MDS responds poorly to the various kinds of drugs used to treat anemia, and in the past it was called refractory anemia. Moreover, 25% to 40% of MDS patients develop acute leukemia, so MDS has also been referred to as preleukemia or a preleukemic condition. When blood diseases are classified as either erythrocytic or leukocytic, it is often unclear into which category MDS falls. Although MDS sometimes occurs in young adults and children, it most often appears in older patients. Diagnosis is confirmed in laboratory tests by a reduction in peripheral blood cells, an abund

In [10]:
# STEP 4 do a synthesis of all summaries to answer question: 
synthesis =   nrpm.synthesize_all_articles(article_summaries, QUESTION)
print("synthesis")
print(synthesis)

synthesis
('Literature Summary: Chronic neutropenia in children can have various etiologies, including autoimmune neutropenia of infancy and primary autoimmune neutropenia in adults. Antineutrophil antibody testing and quantification may aid in diagnosing chronic neutropenia, with studies suggesting its potential value in predicting the clinical course of the disease. Additionally, investigations such as bone marrow aspiration and biopsy, steroid stimulation tests, and in vitro assays like CFU-GM and CSA can help classify neutropenia and guide prognosis. Treatment with granulocyte colony-stimulating factor (G-CSF) has shown efficacy in increasing blood neutrophil counts, particularly in patients with recurrent fevers, inflammatory symptoms, and infections. However, the evidence is primarily based on observational studies with small sample sizes, limiting the generalizability of these findings and highlighting the need for further research with stronger study designs.\n\nTL;DR: Antineut

# Great! We answered our first question using Clinfo.AI!
## Here are all the steps condensed:

In [11]:
import pandas as pd
mpg = pd.read_csv(os.path.join("PubMedRS-200","PubMedRS-200.csv"))
mpg.head()
# mpg.columns

,specialty,SubTopic,Title,Abstract,Introduction,Methods,Results,Conclusion,PMID,Ref_PMIDs,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,allergy and immunology,asthma,Are prenatal anxiety or depression symptoms as...,BACKGROUND:\nAsthma is the most common respira...,Asthma is the most common respiratory disease ...,"PubMed, Embase, and theÂ Cochrane Library were...",A total of 598 studies were initially identifi...,This meta-analysis demonstrated that prenatal ...,34158009,"['18158379', '17932381', '21094921', '21474570...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,allergy and immunology,HIV,Should we care about Plasmodium vivax and HIV ...,BACKGROUND:\nMalaria and HIV are two important...,Malaria and HIV are two important public healt...,Medical records from a tertiary care centre in...,"A total of 1,048 vivax malaria patients were h...",Reports of HIV/PvCo are scarce in the literatu...,33407474,"['23327493', '23327493', '22970336', '27402513...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,allergy and immunology,HIV,Bladder Cancer in HIV-infected Adults: An Emer...,OBJECTIVES:\nNon-AIDS-related malignancies now...,Non-AIDS-related malignancies now represent a ...,We conducted a single center retrospective stu...,During the study period we identified 15 HIV-i...,Bladder cancers in HIV-infected patients remai...,26642314,"['24901259', '19741479', '19818686', '19219610...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,allergy and immunology,HIV,Is having sex with other men a risk factor for...,BACKGROUND:\nAlthough increased prevalence of ...,Although increased prevalence of transfusion t...,"We searched MEDLINE, Embase, The Cochrane Cent...","Out of 18 987 articles, 14 observational studi...",High-quality studies investigating the risk of...,25875812,"['24498030', '24498030', '19638153', '20527321...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,allergy and immunology,pediatric allergy,"Atopic dermatitis, atopic eczema, or eczema? A...",BACKGROUND:\nThe lack of standardized nomencla...,The lack of standardized nomenclature for atop...,"A systematic review of the MEDLINE, EMBASE, an...","In MEDLINE, 33 060 were identified, of which 2...",Atopic dermatitis is the most commonly used te...,27392131,"['14657842', '14657842', '16867052', '26538253...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
clean_data =  mpg.iloc[:, :15]
clean_data.head()

,specialty,SubTopic,Title,Abstract,Introduction,Methods,Results,Conclusion,PMID,Ref_PMIDs,Ref_DOIs,questions,PublishedDate,HumanQuestions,HumanAnswer
0,allergy and immunology,asthma,Are prenatal anxiety or depression symptoms as...,BACKGROUND:\nAsthma is the most common respira...,Asthma is the most common respiratory disease ...,"PubMed, Embase, and theÂ Cochrane Library were...",A total of 598 studies were initially identifi...,This meta-analysis demonstrated that prenatal ...,34158009,"['18158379', '17932381', '21094921', '21474570...","['10.1097/PSY.0b013e31815c1b71', '10.1164/rccm...",Are prenatal anxiety or depression symptoms as...,11/1/2021,Are prenatal anxiety or depression symptoms as...,prenatal mental disorders increase the risk of...
1,allergy and immunology,HIV,Should we care about Plasmodium vivax and HIV ...,BACKGROUND:\nMalaria and HIV are two important...,Malaria and HIV are two important public healt...,Medical records from a tertiary care centre in...,"A total of 1,048 vivax malaria patients were h...",Reports of HIV/PvCo are scarce in the literatu...,33407474,"['23327493', '23327493', '22970336', '27402513...","['10.1186/1756-3305-6-18', '10.1186/1756-3305-...",Should we care about Plasmodium vivax and HIV ...,7/9/2021,Should we care about Plasmodium vivax and HIV ...,Reports of HIV/PvCo are scarce in the literatu...
2,allergy and immunology,HIV,Bladder Cancer in HIV-infected Adults: An Emer...,OBJECTIVES:\nNon-AIDS-related malignancies now...,Non-AIDS-related malignancies now represent a ...,We conducted a single center retrospective stu...,During the study period we identified 15 HIV-i...,Bladder cancers in HIV-infected patients remai...,26642314,"['24901259', '19741479', '19818686', '19219610...","['10.1097/QAD.0000000000000222', '10.1097/QAD....",Bladder Cancer in HIV-infected Adults: An Emer...,6/17/2016,Bladder Cancer in HIV-infected Adults common?,Bladder cancer was diagnosed a median of 14 ye...
3,allergy and immunology,HIV,Is having sex with other men a risk factor for...,BACKGROUND:\nAlthough increased prevalence of ...,Although increased prevalence of transfusion t...,"We searched MEDLINE, Embase, The Cochrane Cent...","Out of 18 987 articles, 14 observational studi...",High-quality studies investigating the risk of...,25875812,"['24498030', '24498030', '19638153', '20527321...","['10.1371/journal.pone.0087139', '10.1371/jour...",Is having sex with other men a risk factor for...,1/15/2016,Is having sex with other men a risk factor for...,High-quality studies investigating the risk of...
4,allergy and immunology,pediatric allergy,"Atopic dermatitis, atopic eczema, or eczema? A...",BACKGROUND:\nThe lack of standardized nomencla...,The lack of standardized nomenclature for atop...,"A systematic review of the MEDLINE, EMBASE, an...","In MEDLINE, 33 060 were identified, of which 2...",Atopic dermatitis is the most commonly used te...,27392131,"['14657842', '14657842', '16867052', '26538253...",[],"Atopic dermatitis, atopic eczema, or eczema?",11/16/2017,What is the most common term Atopic dermatitis...,Atopic dermatitis is the most commonly used te...


In [31]:
from enum import Enum

class EvaluationType(Enum):
    ResSearch = "RestrictedSearch"
    SrcDropped = "SourcedDropped"
    UnresSearch= "UnrestrictedSearch"


def filter_article_res_search ():
    return clean_data[clean_data["HumanAnswer"] == "Yes"]

def filter_article_drop_source (article_ids, row):
    pmid = row["PMID"]

    if pmid in article_ids:
        print(f"Found the article, Removing {pmid}")
        article_ids.remove(pmid)
    return article_ids

    

def construct_response_csv (evaluationType: EvaluationType, data: pd.DataFrame, isVietnamese: bool = False):
    response ={}
    for column in clean_data.columns:
        response[column] = []

    response["Summary"] = []
    response["TLDR"] = []
    response["All"] = []

    for index, row in data.iterrows():
        question = row["HumanQuestions"]

        if isVietnamese:
            question = nrpm.translate_vn_to_en(question)

        try:
            pubmed_queries, article_ids = nrpm.search_pubmed(question,num_results=10,num_query_attempts=1)

            if evaluationType == EvaluationType.ResSearch:
                article_ids = filter_article_res_search()
            elif evaluationType == EvaluationType.SrcDropped:
                article_ids = filter_article_drop_source(article_ids, row)


            articles = nrpm.fetch_article_data(article_ids)
            article_summaries,irrelevant_articles =  nrpm.summarize_each_article(articles, question)
            synthesis =   nrpm.synthesize_all_articles(article_summaries, question)

            #extract data from synthesis tuple
            synthesis = synthesis[0]
            if (isVietnamese):
                synthesis = nrpm.translate_en_to_vn(synthesis)
            sentences = synthesis.split("\n")
            combined =""
            for sentence in sentences:
                if "Literature Summary: " in sentence or  "Tóm tắt tài liệu: "  in sentence:
                    combined += sentence
                    synthesis_only = sentence.replace("Literature Summary: ", "")
                    synthesis_only = sentence.replace("Tóm tắt tài liệu: ", "")
                    response["Summary"].append(synthesis_only)
                elif "TL;DR" in sentence or "Tóm tắt:"  in sentence:
                    combined += "\n" + sentence
                    TLDR = sentence.replace("TL;DR:", "")
                    TLDR = sentence.replace("Tóm tắt: ", "")
                    response["TLDR"].append(TLDR)
                    break

            response["All"].append(combined)  

            for column in clean_data.columns:
                response[column].append(row[column])

            if index == 10:
                break
        except Exception as e:
            print(f"Error: {e}")
            continue

    
    csv_name = ""
    if isVietnamese:
        csv_name = f"VN_{evaluationType.value}.csv"
    else:
        csv_name = f"{evaluationType.value}.csv"    
    response_df = pd.DataFrame(response)
    response_df.to_csv(csv_name, mode='w', index=False)



In [35]:
construct_response_csv(EvaluationType.UnresSearch, clean_data)
RestrictedSearch = pd.read_csv("UnrestrictedSearch.csv")
RestrictedSearch.head()

Could not find 'Abstract' for article with PMID = 36161367
Could not find 'Abstract' for article with PMID = 36648466
Could not find 'Abstract' for article with PMID = 36280628


,specialty,SubTopic,Title,Abstract,Introduction,Methods,Results,Conclusion,PMID,Ref_PMIDs,Ref_DOIs,questions,PublishedDate,HumanQuestions,HumanAnswer,Summary,TLDR,All
0,allergy and immunology,asthma,Are prenatal anxiety or depression symptoms as...,BACKGROUND:\nAsthma is the most common respira...,Asthma is the most common respiratory disease ...,"PubMed, Embase, and theÂ Cochrane Library were...",A total of 598 studies were initially identifi...,This meta-analysis demonstrated that prenatal ...,34158009,"['18158379', '17932381', '21094921', '21474570...","['10.1097/PSY.0b013e31815c1b71', '10.1164/rccm...",Are prenatal anxiety or depression symptoms as...,11/1/2021,Are prenatal anxiety or depression symptoms as...,prenatal mental disorders increase the risk of...,Literature Summary: Prenatal maternal depressi...,TL;DR: The evidence suggests a potential assoc...,Literature Summary: Prenatal maternal depressi...
1,allergy and immunology,HIV,Should we care about Plasmodium vivax and HIV ...,BACKGROUND:\nMalaria and HIV are two important...,Malaria and HIV are two important public healt...,Medical records from a tertiary care centre in...,"A total of 1,048 vivax malaria patients were h...",Reports of HIV/PvCo are scarce in the literatu...,33407474,"['23327493', '23327493', '22970336', '27402513...","['10.1186/1756-3305-6-18', '10.1186/1756-3305-...",Should we care about Plasmodium vivax and HIV ...,7/9/2021,Should we care about Plasmodium vivax and HIV ...,Reports of HIV/PvCo are scarce in the literatu...,Literature Summary: Plasmodium vivax and HIV c...,TL;DR: While evidence suggests a higher risk o...,Literature Summary: Plasmodium vivax and HIV c...
2,allergy and immunology,HIV,Bladder Cancer in HIV-infected Adults: An Emer...,OBJECTIVES:\nNon-AIDS-related malignancies now...,Non-AIDS-related malignancies now represent a ...,We conducted a single center retrospective stu...,During the study period we identified 15 HIV-i...,Bladder cancers in HIV-infected patients remai...,26642314,"['24901259', '19741479', '19818686', '19219610...","['10.1097/QAD.0000000000000222', '10.1097/QAD....",Bladder Cancer in HIV-infected Adults: An Emer...,6/17/2016,Bladder Cancer in HIV-infected Adults common?,Bladder cancer was diagnosed a median of 14 ye...,Literature Summary: Bladder cancer in HIV-infe...,TL;DR: Bladder cancer in HIV-infected adults i...,Literature Summary: Bladder cancer in HIV-infe...
3,allergy and immunology,HIV,Is having sex with other men a risk factor for...,BACKGROUND:\nAlthough increased prevalence of ...,Although increased prevalence of transfusion t...,"We searched MEDLINE, Embase, The Cochrane Cent...","Out of 18 987 articles, 14 observational studi...",High-quality studies investigating the risk of...,25875812,"['24498030', '24498030', '19638153', '20527321...","['10.1371/journal.pone.0087139', '10.1371/jour...",Is having sex with other men a risk factor for...,1/15/2016,Is having sex with other men a risk factor for...,High-quality studies investigating the risk of...,Literature Summary: The available evidence fro...,TL;DR: There is uncertain evidence suggesting ...,Literature Summary: The available evidence fro...
4,allergy and immunology,pediatric allergy,"Atopic dermatitis, atopic eczema, or eczema? A...",BACKGROUND:\nThe lack of standardized nomencla...,The lack of standardized nomenclature for atop...,"A systematic review of the MEDLINE, EMBASE, an...","In MEDLINE, 33 060 were identified, of which 2...",Atopic dermatitis is the most commonly used te...,27392131,"['14657842', '14657842', '16867052', '26538253...",[],"Atopic dermatitis, atopic eczema, or eczema?",11/16/2017,What is the most common term Atopic dermatitis...,Atopic dermatitis is the most commonly used te...,Literature Summary: The term recommended as th...,TL;DR: The most common term recommended is ecz...,Literature Summary: The term recommended as th...


In [15]:
# construct_response_csv(EvaluationType.UnresSearch)
# RestrictedSearch = pd.read_csv("RestrictedSearch.csv")
# RestrictedSearch.head()

In [16]:
# construct_response_csv(EvaluationType.UnresSearch)
# RestrictedSearch = pd.read_csv("RestrictedSearch.csv")
# RestrictedSearch.head()

# VIETNAMESE TRANSLATION

In [24]:
headers = clean_data.columns.tolist()
vn_data = pd.read_csv("ViPubMedRS-200.csv")
vn_clean_data =  vn_data.iloc[:, :15]
vn_clean_data.to_csv("ViPubMedRS-200.with_column.csv", mode='w', index=False, header=headers)

In [25]:
vn_clean_data = pd.read_csv("ViPubMedRS-200.with_column.csv")
vn_clean_data.head()

,specialty,SubTopic,Title,Abstract,Introduction,Methods,Results,Conclusion,PMID,Ref_PMIDs,Ref_DOIs,questions,PublishedDate,HumanQuestions,HumanAnswer
0,dị ứng và miễn dịch học,HIV,Chúng ta có nên quan tâm đến sự đồng nhiễm Pla...,NỀN TẢNG:\nSốt rét và HIV là hai vấn đề sức kh...,Sốt rét và HIV là hai vấn đề quan trọng về sức...,Hồ sơ y tế từ một trung tâm chăm sóc thứ ba ở ...,Tổng cộng có 1.048 bệnh nhân sốt rét vivax đã ...,Các báo cáo về HIV/PvCo rất hiếm trong tài liệ...,33407474,"['23327493', '23327493', '22970336', '27402513...","['10.1186/1756-3305-6-18', '10.1186/1756-3305-...",Chúng ta có nên quan tâm đến sự đồng nhiễm Pla...,7/9/2021,Chúng ta có nên quan tâm đến sự đồng nhiễm Pla...,Các báo cáo về HIV/PvCo rất hiếm trong tài liệ...
1,dị ứng và miễn dịch học,HIV,Ung thư bàng quang ở người lớn nhiễm HIV: Một ...,MỤC TIÊU:\nCác khối u ác tính không liên quan ...,Các khối u ác tính không liên quan đến AIDS hi...,Chúng tôi đã tiến hành một nghiên cứu hồi cứu ...,"Trong thời gian nghiên cứu, chúng tôi đã xác đ...",Ung thư bàng quang ở bệnh nhân nhiễm HIV vẫn h...,26642314,"['24901259', '19741479', '19818686', '19219610...","['10.1097/QAD.0000000000000222', '10.1097/QAD....",Ung thư bàng quang ở người lớn nhiễm HIV: Một ...,6/17/2016,Ung thư bàng quang ở người lớn nhiễm HIV có ph...,Ung thư bàng quang được chẩn đoán trung bình 1...
2,dị ứng và miễn dịch học,HIV,Có phải quan hệ tình dục với những người đàn ô...,BỐI CẢNH:\nMặc dù sự gia tăng tỷ lệ nhiễm trùn...,Mặc dù sự gia tăng tỷ lệ nhiễm trùng lây truyề...,"Chúng tôi đã tìm kiếm MEDLINE, Embase, The Coc...","Trong số 18.987 bài báo, có 14 nghiên cứu quan...",Các nghiên cứu chất lượng cao điều tra nguy cơ...,25875812,"['24498030', '24498030', '19638153', '20527321...","['10.1371/journal.pone.0087139', '10.1371/jour...",Việc quan hệ tình dục với những người đàn ông ...,1/15/2016,Việc quan hệ tình dục với những người đàn ông ...,Các nghiên cứu chất lượng cao điều tra nguy cơ...
3,dị ứng và miễn dịch học,dị ứng nhi khoa,"Viêm da dị ứng, chàm dị ứng, hay chàm? Một đán...",NỀN TẢNG:\nSự thiếu hụt về hệ thống đặt tên ch...,Sự thiếu hụt hệ thống đặt tên chuẩn cho bệnh c...,"Một đánh giá hệ thống về MEDLINE, EMBASE và LI...","Trong MEDLINE, có 33.060 tài liệu được xác địn...",Viêm da dị ứng là thuật ngữ được sử dụng phổ b...,27392131,"['14657842', '14657842', '16867052', '26538253...",[],"Viêm da dị ứng, chàm dị ứng, hay chàm?",11/16/2017,"Thuật ngữ phổ biến nhất là viêm da dị ứng, viê...",Viêm da dị ứng là thuật ngữ được sử dụng phổ b...
4,dị ứng và miễn dịch học,dị ứng nhi khoa,Điều trị hen suyễn bằng axit béo omega-3: bằng...,NỀN TẢNG:\nCó sự quan tâm đáng kể đến giá trị ...,Có sự quan tâm đáng kể đến giá trị điều trị ti...,"Medline, Premedline, Embase, Cochrane Central ...",Mười RCT đã được tìm thấy có liên quan đến báo...,Với bức tranh không nhất quán chủ yếu trong và...,16854238,"['15256393', '15256393', '12442909', '11172168...","['10.1164/rccm.200405-611PP', '10.1164/rccm.20...",Điều trị hen suyễn bằng axit béo omega-3: bằng...,9/27/2006,Điều trị hen suyễn bằng axit béo omega-3: bằng...,Với bức tranh không nhất quán chủ yếu trong và...


In [32]:
construct_response_csv(EvaluationType.UnresSearch, vn_clean_data, isVietnamese=True)

Could not find 'Abstract' for article with PMID = 2182139
Could not find 'Abstract' for article with PMID = 7857558
Could not find 'Abstract' for article with PMID = 3058251
Error: Supplied id parameter is empty.
Could not find 'Abstract' for article with PMID = 29874694
Could not find 'Abstract' for article with PMID = 29111813


In [33]:
vn_data= pd.read_csv("VN_UnrestrictedSearch.csv")
vn_data.head()

,specialty,SubTopic,Title,Abstract,Introduction,Methods,Results,Conclusion,PMID,Ref_PMIDs,Ref_DOIs,questions,PublishedDate,HumanQuestions,HumanAnswer,Summary,TLDR,All
0,dị ứng và miễn dịch học,HIV,Chúng ta có nên quan tâm đến sự đồng nhiễm Pla...,NỀN TẢNG:\nSốt rét và HIV là hai vấn đề sức kh...,Sốt rét và HIV là hai vấn đề quan trọng về sức...,Hồ sơ y tế từ một trung tâm chăm sóc thứ ba ở ...,Tổng cộng có 1.048 bệnh nhân sốt rét vivax đã ...,Các báo cáo về HIV/PvCo rất hiếm trong tài liệ...,33407474,"['23327493', '23327493', '22970336', '27402513...","['10.1186/1756-3305-6-18', '10.1186/1756-3305-...",Chúng ta có nên quan tâm đến sự đồng nhiễm Pla...,7/9/2021,Chúng ta có nên quan tâm đến sự đồng nhiễm Pla...,Các báo cáo về HIV/PvCo rất hiếm trong tài liệ...,Các nghiên cứu kiểm tra tình trạng đồng nhiễm ...,TL;DR: Bằng chứng cho thấy nhiễm trùng đồng th...,Tóm tắt tài liệu: Các nghiên cứu kiểm tra tình...
1,dị ứng và miễn dịch học,HIV,Ung thư bàng quang ở người lớn nhiễm HIV: Một ...,MỤC TIÊU:\nCác khối u ác tính không liên quan ...,Các khối u ác tính không liên quan đến AIDS hi...,Chúng tôi đã tiến hành một nghiên cứu hồi cứu ...,"Trong thời gian nghiên cứu, chúng tôi đã xác đ...",Ung thư bàng quang ở bệnh nhân nhiễm HIV vẫn h...,26642314,"['24901259', '19741479', '19818686', '19219610...","['10.1097/QAD.0000000000000222', '10.1097/QAD....",Ung thư bàng quang ở người lớn nhiễm HIV: Một ...,6/17/2016,Ung thư bàng quang ở người lớn nhiễm HIV có ph...,Ung thư bàng quang được chẩn đoán trung bình 1...,Một nghiên cứu lớn dựa trên dân số của Silverb...,Bằng chứng cho thấy mối liên hệ tiềm ẩn giữa u...,Tóm tắt tài liệu: Một nghiên cứu lớn dựa trên ...
2,dị ứng và miễn dịch học,HIV,Có phải quan hệ tình dục với những người đàn ô...,BỐI CẢNH:\nMặc dù sự gia tăng tỷ lệ nhiễm trùn...,Mặc dù sự gia tăng tỷ lệ nhiễm trùng lây truyề...,"Chúng tôi đã tìm kiếm MEDLINE, Embase, The Coc...","Trong số 18.987 bài báo, có 14 nghiên cứu quan...",Các nghiên cứu chất lượng cao điều tra nguy cơ...,25875812,"['24498030', '24498030', '19638153', '20527321...","['10.1371/journal.pone.0087139', '10.1371/jour...",Việc quan hệ tình dục với những người đàn ông ...,1/15/2016,Việc quan hệ tình dục với những người đàn ông ...,Các nghiên cứu chất lượng cao điều tra nguy cơ...,Một nghiên cứu của Allain và cộng sự (2002) đư...,Quan hệ tình dục với những người đàn ông khác ...,Tóm tắt tài liệu: Một nghiên cứu của Allain và...
3,dị ứng và miễn dịch học,dị ứng nhi khoa,"Viêm da dị ứng, chàm dị ứng, hay chàm? Một đán...",NỀN TẢNG:\nSự thiếu hụt về hệ thống đặt tên ch...,Sự thiếu hụt hệ thống đặt tên chuẩn cho bệnh c...,"Một đánh giá hệ thống về MEDLINE, EMBASE và LI...","Trong MEDLINE, có 33.060 tài liệu được xác địn...",Viêm da dị ứng là thuật ngữ được sử dụng phổ b...,27392131,"['14657842', '14657842', '16867052', '26538253...",[],"Viêm da dị ứng, chàm dị ứng, hay chàm?",11/16/2017,"Thuật ngữ phổ biến nhất là viêm da dị ứng, viê...",Viêm da dị ứng là thuật ngữ được sử dụng phổ b...,Một nghiên cứu lớn dựa trên dân số của Silverb...,Bằng chứng cho thấy thuật ngữ phổ biến nhất đư...,Tóm tắt tài liệu: Một nghiên cứu lớn dựa trên ...
4,dị ứng và miễn dịch học,dị ứng nhi khoa,Điều trị hen suyễn bằng axit béo omega-3: bằng...,NỀN TẢNG:\nCó sự quan tâm đáng kể đến giá trị ...,Có sự quan tâm đáng kể đến giá trị điều trị ti...,"Medline, Premedline, Embase, Cochrane Central ...",Mười RCT đã được tìm thấy có liên quan đến báo...,Với bức tranh không nhất quán chủ yếu trong và...,16854238,"['15256393', '15256393', '12442909', '11172168...","['10.1164/rccm.200405-611PP', '10.1164/rccm.20...",Điều trị hen suyễn bằng axit béo omega-3: bằng...,9/27/2006,Điều trị hen suyễn bằng axit béo omega-3: bằng...,Với bức tranh không nhất quán chủ yếu trong và...,Một phân tích tổng hợp của Zheng và cộng sự (2...,Axit béo omega-3 có thể cải thiện chức năng ph...,Tóm tắt tài liệu: Một phân tích tổng hợp của Z...
